# Reference
- dcm2nifti & change file name : Thanks to 강민석 교수님.. 
- Denoising process: https://github.com/cosanlab/vmPFC_dynamics/blob/master/Notebooks/1_VMPFC_Denoise.ipynb

# 0. Make a BIDS format 

In [ ]:
# Change fieldmap name with shell script !                 
path = '/mnt/c/Users/since/Desktop/Kdrama_prac/bash_create_folders_in_nifti.sh' #파일이 존재하는 로컬 경로 설정. 스크립트 들어가서 먼저 변수 설정 바꿔주기.. 
sp.call(path, shell=True)

# 1. dcm2nifti : Conver dcm data to nifti

In [2]:
from nipype.interfaces.dcm2nii import Dcm2niix
import nipype.interfaces.io as nio
import warnings
import subprocess as sp

warnings.filterwarnings(action='ignore')

In [3]:
subjsource = nio.DataGrabber()

In [9]:
subjsource.inputs.base_directory = '/mnt/c/Users/since/Desktop/Kdrama_prac/dicom' # make sure to change output directory below
#subjsource.inputs.template = 'S1[0-5]'
subjsource.inputs.template = 'S1[4-6]'
subjsource.inputs.sort_filelist = True
subjresults = subjsource.run()

converter = Dcm2niix()

if isinstance(subjresults.outputs.outfiles,str):
    subjresults.outputs.outfiles = [subjresults.outputs.outfiles]
    
print(subjresults.outputs.outfiles)

['/mnt/c/Users/since/Desktop/Kdrama_prac/dicom/S14', '/mnt/c/Users/since/Desktop/Kdrama_prac/dicom/S15', '/mnt/c/Users/since/Desktop/Kdrama_prac/dicom/S16']


In [10]:
# Convert Dicom to nifti : sub- 11 & 15 
for cur_subj in subjresults.outputs.outfiles:
    cur_len = len(cur_subj)
    cur_sid = 'sub-' + cur_subj[cur_len-2:cur_len]
    res_dir = cur_subj[0:cur_len-9]+'data/'
    print(res_dir)
    print('%s' %cur_sid)
    
    #for cur_type in ['*ENCODING*','*RECALL*','T2*','T1*','GRE*','PA*']:
    for cur_type in ['ENCODING*', 'RECALL*', 'T1*', 'GRE*', 'PA*']:       
        if cur_type == 'T1*' or cur_type == 'T2*':
            cur_subdir = '/anat'
        elif cur_type == 'GRE*' or cur_type == 'PA*':
            cur_subdir = '/fmap'
        else: 
            cur_subdir = '/func'
            
        #Grab Relevant Data
        datasource = nio.DataGrabber()
        datasource.inputs.base_directory = cur_subj
        datasource.inputs.template = cur_type
        datasource.inputs.sort_filelist = True
        datasource.inputs.raise_on_empty = False
        results = datasource.run()
        
        idx = 1
        if results.outputs.outfiles is not None:
            if isinstance(results.outputs.outfiles,str):
                results.outputs.outfiles = [results.outputs.outfiles]

        for cur_file in results.outputs.outfiles:
                print('cur_file is', cur_file)
                
                # set output file name
                if cur_type == 'T1*':
                    cur_output_filename = cur_sid + '_T1w'
                #elif cur_type == 'T2*':
                #    cur_output_filename = cur_sid + '_T2w'
                elif cur_type == 'GRE*':
                    cur_output_filename = cur_sid + '_fmap'
                elif cur_type == 'ENCODING*':
                    cur_output_filename = cur_sid + '_task-encoding_run-' + str(idx) + '_bold'
                elif cur_type == 'RECALL*':
                    cur_output_filename = cur_sid + '_task-recall_run-' + str(idx) + '_bold'
                elif cur_type == 'PA*':
                    cur_output_filename = cur_sid + '_dir-pa' + '_epi'
                              
                converter.inputs.source_dir = cur_file
                converter.inputs.compression = 5
                converter.inputs.bids_format = True
                converter.inputs.output_dir = res_dir + cur_sid + cur_subdir 
                converter.inputs.out_filename = cur_output_filename
                converter.run()
                idx = idx + 1

                print(converter.inputs.output_dir + '/' + converter.inputs.out_filename)

/mnt/c/Users/since/Desktop/Kdrama_prac/data/
sub-14
cur_file is /mnt/c/Users/since/Desktop/Kdrama_prac/dicom/S14/ENCODING-1_0009
230712-12:06:13,366 nipype.interface INFO:
	 stdout 2023-07-12T12:06:13.366834:Chris Rorden's dcm2niiX version v1.0.20230411  GCC12.2.0 x86-64 (64-bit Linux)
230712-12:06:13,367 nipype.interface INFO:
	 stdout 2023-07-12T12:06:13.366834:Found 477 DICOM file(s)
230712-12:06:13,368 nipype.interface INFO:
	 stdout 2023-07-12T12:06:13.366834:Convert 477 DICOM as /mnt/c/Users/since/Desktop/Kdrama_prac/data/sub-14/func/sub-14_task-encoding_run-1_bold (80x80x48x477)
230712-12:06:14,521 nipype.interface INFO:
	 stdout 2023-07-12T12:06:14.521013:Compress: "/usr/bin/pigz" -b 960 -n -f -5 "/mnt/c/Users/since/Desktop/Kdrama_prac/data/sub-14/func/sub-14_task-encoding_run-1_bold.nii"
230712-12:06:14,521 nipype.interface INFO:
	 stdout 2023-07-12T12:06:14.521013:Conversion required 12.556428 seconds (1.988587 for core code).
/mnt/c/Users/since/Desktop/Kdrama_prac/data/sub-1

sub-15_fmap_e1.json not renamed: sub-15_magnitude1.json already exists
sub-15_fmap_e1.nii.gz not renamed: sub-15_magnitude1.nii.gz already exists
sub-15_fmap_e2_ph.json not renamed: sub-15_phasediff.json already exists
sub-15_fmap_e2_ph.nii.gz not renamed: sub-15_phasediff.nii.gz already exists
sub-15_fmap_e2.json not renamed: sub-15_magnitude2.json already exists
sub-15_fmap_e2.nii.gz not renamed: sub-15_magnitude2.nii.gz already exists


0

In [13]:
                
# Change fieldmap name with shell script !                 
path = '/mnt/c/Users/since/Desktop/Kdrama_prac/bash_change_fieldmap_name_May.sh' #파일이 존재하는 로컬 경로 설정 
sp.call(path, shell=True)

/mnt/c/Users/since/Desktop/Kdrama_prac/data/sub-14
/mnt/c/Users/since/Desktop/Kdrama_prac/data/sub-15
/mnt/c/Users/since/Desktop/Kdrama_prac/data/sub-16


sub-15_fmap_e1.json not renamed: sub-15_magnitude1.json already exists
sub-15_fmap_e1.nii.gz not renamed: sub-15_magnitude1.nii.gz already exists
sub-15_fmap_e2.json not renamed: sub-15_magnitude2.json already exists
sub-15_fmap_e2.nii.gz not renamed: sub-15_magnitude2.nii.gz already exists


0

In [12]:
cur_subj[0:cur_len-9]

'/mnt/c/Users/since/Desktop/Kdrama_prac/dic'

# 2. Run fMRIprep 

# 3. (Optional but necessary..) Denoising 

In [2]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltools.stats import regress, zscore
from nltools.data import Brain_Data, Design_Matrix
from nltools.stats import find_spikes 
from nltools.mask import expand_mask

base_dir = '/mnt/c/Users/since/Desktop/Kdrama_prac/'
data_dir = os.path.join(base_dir, 'data2', 'derivatives')
denoised_dir = os.path.join(base_dir, 'analysis', 'denoised')

/home/sincerely/.conda/envs/fmri01/lib/python3.7/site-packages/nilearn/__init__.py:69: FutureWarning: Python 3.7 support is deprecated and will be removed in release 0.12 of Nilearn. Consider switching to Python 3.9 or 3.10.
  _python_deprecation_warnings()


In [34]:
subjects = ['14','15', '16'] 
fwhm = 6
tr = 1.5
outlier_cutoff = 3

In [35]:
for s in subjects:
    print(s)
    file_list = glob.glob(os.path.join(data_dir, f'sub-{s}', 'func', '*preproc*.nii.gz'))
    print(len(file_list))
    for f in file_list:
        sub = os.path.basename(f).split('_')[0]
        task = os.path.basename(f).split('_')[1]
        run = os.path.basename(f).split('_')[2]
           
        # if not os.path.exists(result_dir):
        #     os.makedirs(result_dir)
        csf = Brain_Data(glob.glob(os.path.join(data_dir, f'{sub}', 'anat', '*CSF*.nii.gz'))[1])
        data = Brain_Data(f) #denoise 
        smoothed = data.smooth(fwhm=fwhm)   # Spatial smoothing 
        
        spikes = smoothed.find_spikes(global_spike_cutoff=outlier_cutoff, diff_spike_cutoff=outlier_cutoff) #FD가 2std 초과하는 것들만 Outlier로 만들었다. 
        covariates = pd.read_csv(glob.glob(os.path.join(data_dir, sub, 'func', f'{sub}_{task}_{run}_desc-confounds_timeseries.tsv'))[0], sep='\t') #Motion params !! 
        
        mc = covariates[['trans_x','trans_y','trans_z','rot_x', 'rot_y', 'rot_z']]
        mc_cov = make_motion_covariates(mc, tr)
        motionOutliers = covariates.filter(regex='motion_outlier')
        
        csf = zscore(pd.Series(smoothed.extract_roi(csf.threshold(upper=0.85, binarize=True)))) # Use CSF from fmriprep output
        
        dm = Design_Matrix(pd.concat([csf, mc_cov, motionOutliers, spikes.drop(labels='TR', axis=1)], axis=1), sampling_freq=1/tr)
        dm = dm.add_poly(order=2, include_lower=True) # Add Intercept, Linear and Quadratic Trends

        smoothed.X = dm 
        stats = smoothed.regress()
        stats['residual'].data = np.float32(stats['residual'].data) # cast as float32 to reduce storage space
        stats['residual'].write(os.path.join(denoised_dir, f'{sub}_denoise_{task}_{run}_space-MNI152NLin6Asym_desc-preproc_bold.nii.gz'))
        print( f'finisih_{sub}_{task}_{run}_data')

14
4


KeyboardInterrupt: 

In [ ]:
# Make hdf5 for efficient calculation 
file_list = glob.glob(os.path.join(denoised_dir, '*.nii.gz'))
#print(file_list)
for f in file_list:
    data = Brain_Data(f)
    data.write(f"{f.split('.nii.gz')[0]}.hdf5")
    print('finish'+f"{f.split('.nii.gz')[0]}.hdf5")

In [ ]:
# Average for each ROI 
mask = Brain_Data('http://neurovault.org/media/images/2099/Neurosynth%20Parcellation_0.nii.gz') #3iso 짜리, Dartbrain에서 사용한 ROI인데 cortex 쪽에만 존재한다. 
#mask = Brain_Data(mask_path)
mask_x = expand_mask(mask)

file_list = glob.glob(os.path.join(denoised_dir, f'*.hdf5'))
for f in file_list:
    sub = os.path.basename(f).split('_')[0]
    task = os.path.basename(f).split('_')[2]
    run = os.path.basename(f).split('_')[3]    
    data = Brain_Data(f)
    # for i,m in enumerate(mask_x):  #for masking each ROI
    #     roi = data.apply_mask(m)
    #     pd.DataFrame(roi.data).to_csv(os.path.join(denoised_dir, 'ROIs', f'{sub}_{task}_{run}_denoised_roi{i}.csv' ), index=False)
    avg_roi = data.extract_roi(mask)
    pd.DataFrame(avg_roi.T).to_csv(os.path.join(denoised_dir, 'Averaged_ROI', f'{sub}_{task}_{run}_Average_ROI_n50.csv'), index=False)
    print(f'finish_{sub}_{task}_{run}') 